In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rp
import gmaps
from config import gkey
import scipy.stats as stats

In [48]:
base = "http://ufo-api.herokuapp.com/api/sightings/location/near?"
Ontario_lat_max = 56.800878
Ontario_lat_min = 41.771312
Ontario_lng_max = -75.251467
Ontario_lng_min = -95.281572

lats = np.random.uniform(low=Ontario_lat_min, high=Ontario_lat_max, size=2000)
lngs = np.random.uniform(low=Ontario_lng_min, high=Ontario_lng_max, size=2000)


In [49]:
len(lats),len(lngs)

(2000, 2000)

In [50]:
city = []
shape = []
state = []
date = []
duration = []
lats_ON = []
lngs_ON = []
for i in np.arange(0,len(lats)):
    try:
        responce = rp.get(f"{base}lat={lats[i]}&lon={lngs[i]}&limit=1").json()
        if responce["sightings"][0]["obj"]["state"] == "ON":
            city.append(responce["sightings"][0]["obj"]["city"])
            shape.append(responce["sightings"][0]["obj"]["shape"])
            state.append(responce["sightings"][0]["obj"]["state"])
            date.append(responce["sightings"][0]["obj"]["date"])
            duration.append(responce["sightings"][0]["obj"]["duration"])
            lats_ON.append(responce["sightings"][0]["obj"]["loc"][1])
            lngs_ON.append(responce["sightings"][0]["obj"]["loc"][0])
    except Exception as e:
        print("could not find")
print("finished")

finished


In [55]:
Combined = pd.DataFrame({"lat":lats_ON,"lng":lngs_ON,"city":city,"state":state,"date":date})
cities_strength = Combined.groupby("city").count()
cities_strength = cities_strength.reset_index()
Total_data = Combined
Combined = Combined.drop_duplicates("city")
cities_strength.set_index("city",inplace = True)
cities_strength = pd.DataFrame(cities_strength.loc[Combined["city"],"lat"])
cities_strength.reset_index(inplace = True)
cities_strength.rename(columns = {"lat":"strength"},inplace=True)
cities_strength["Normalised"] = (cities_strength["strength"]/cities_strength["strength"].sum())*100
Combined = pd.merge(Combined,cities_strength,on="city")
len(Combined),len(Total_data)

(114, 1037)

In [57]:
#cleaning combined dataframe based on date 
Combined['year'] = pd.DatetimeIndex(Combined['date']).year
Combined['month'] = pd.DatetimeIndex(Combined['date']).month
Total_data['year'] = pd.DatetimeIndex(Total_data['date']).year
Total_data['month'] = pd.DatetimeIndex(Total_data['date']).month
# Combined.head()


Combined.to_csv("CSV_files/Ufo-Data.csv",index=False)
Total_data.to_csv("CSV_files/Duplicate-data.csv",index=False)

In [47]:
# gkey = ""
# gmaps.configure(api_key=gkey)
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)

In [9]:

# coordinates = Combined[["lat","lng"]]
# coordinates
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)
# fig
# # Combined[Combined["lat"]<=-50]

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [ ]:

# heat_layer = gmaps.heatmap_layer(coordinates, weights=Combined["strength"], 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 1.0)

# # Adjust heat_layer setting to help with heatmap dissipating on zoom
# heat_layer.dissipating = False
# heat_layer.max_intensity = 1000
# heat_layer.point_radius = 1

# fig.add_layer(heat_layer)

# fig


In [ ]:
# Combined.boxplot("strength", by="shape", figsize=(20, 10))

In [ ]:
# Combined[Combined["shape"]=="Disk"]["strength"]
# plt.hist(Combined[Combined["shape"]=="Disk"]["strength"], 10, density=True, alpha=0.7, label="population1")
# plt.hist(Combined[Combined["shape"]=="Fireball"]["strength"], 10, density=True, alpha=0.7, label="population1")
# plt.hist(Combined[Combined["shape"]=="Circle"]["strength"], 10, density=True, alpha=0.7, label="population1")
# # plt.hist(Combined[Combined["shape"]=="Egg"]["strength"], 10, density=True, alpha=0.7, label="population1")

In [ ]:
# stats.f_oneway(Combined[Combined["shape"]=="Disk"]["strength"],Combined[Combined["shape"]=="Fireball"]["strength"],Combined[Combined["shape"]=="Egg"]["strength"])